In [21]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [22]:
speed_area_rel = pd.read_csv("speed_area_file_1000_custom.csv")

In [23]:
X = speed_area_rel.iloc[:, 2:-1].values
y = speed_area_rel.iloc[:, -1]

In [24]:
speed_area_rel.head()

,driver_id,sp1,sp2,sp3,sp4,sp5,sp6,sp7,sp8,sp9,...,sp993,sp994,sp995,sp996,sp997,sp998,sp999,sp1000,sp1001,area
0,1,0,1,3,6,10,15,21,28,36,...,42,46,45,43,40,36,37,39,42,2240.5
1,2,0,2,6,12,20,30,42,56,72,...,76,74,70,64,56,46,34,36,40,3227.0
2,3,0,3,9,12,18,27,39,42,48,...,57,60,66,75,87,84,78,69,57,3767.0
3,4,0,4,12,24,40,60,84,80,72,...,84,96,92,84,72,56,36,40,48,4407.5
4,5,0,5,15,30,50,55,65,80,100,...,55,75,70,60,45,25,20,10,5,4986.5


In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0, random_state=101)

In [28]:
test_data = pd.read_csv("real_gps.csv")
test_data.rename(columns={"speed(kmph)": "speed"}, inplace=True)
# test_data = test_data[test_data["driver_id"] == 1]
# test_data = test_data.iloc[3710:]
# test_data.reset_index(inplace=True, drop=True)

In [29]:
test_data.tail()

,elapsed_time,latitude,longitutde,speed,new_angle,norm_factor,new_w_speed
1710,1715.333,12.957093,77.743716,8.892,-9.01,0.0,8.89
1711,1716.324,12.957121,77.743720,11.520,-1.61,0.0,11.52
1712,1717.321,12.957153,77.743726,12.168,-1.93,0.0,12.17
1713,1718.316,12.957175,77.743731,6.012,-37.42,0.0,6.01
1714,1719.316,12.957179,77.743735,0.000,0.00,0.0,0.00


In [30]:
check = []
zero = False
for speed in test_data["speed"]:
    if speed > 0:
        check.append(True)
        zero = False
    elif zero is False and speed == 0:
        check.append(True)
        zero = True
    else:
        check.append(False)
test_data = test_data[check]
test_data.reset_index(inplace=True, drop=True)

In [31]:
# test_data = test_data.iloc[600:3709]
# test_data.reset_index(inplace=True, drop=True)

In [32]:
def prediction(model, value):
    pred = model.predict(value)
    return pred

In [33]:
from xgboost import XGBRegressor

In [34]:
xgb = XGBRegressor(random_state=101, booster="gbtree", learning_rate=0.3)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.3, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=101,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [35]:
pred_XGboost = []
interval = []
time = []
run_avg = []
if len(test_data) == 1000:
    pred_XGboost.append(prediction(xgb, test_data.loc[0:1000, "speed"].values.reshape(1, -1))[0])
    interval.append(" - ".join([str(1), str(1000)]))
else:
    for i in range(len(test_data)):
        if 1000+i == len(test_data):
            break
        if i%100 == 0:
            pred_XGboost.append(prediction(xgb, test_data.loc[i+1:1000+i, "speed"].values.reshape(1, -1))[0])
            interval.append(" - ".join([str(i+1), str(1000+i)]))
            time.append(1000+i)
            run_avg.append(np.mean(pred_XGboost))
    
res = pd.DataFrame({"interval": interval, "pred_XGboost": pred_XGboost, "time": time, "run_avg": run_avg})

In [36]:
res.to_csv("prediction_real_data.csv", index=False)

In [37]:
res

,interval,pred_XGboost,run_avg,time
0,1 - 1000,3275.489990,3275.489990,1000
1,101 - 1100,3665.305420,3470.397705,1100
2,201 - 1200,3616.574951,3519.123291,1200
3,301 - 1300,3546.924072,3526.073486,1300
4,401 - 1400,3685.478271,3557.954346,1400
